# Gravix Layer Cookbook: Proactive AI Learning Coach

This notebook is an example cookbook demonstrating the Gravix Layer Memory feature through a Proactive AI Learning Coach use case.

**What you'll learn**:
- How to setup the Gravix Layer client and memory service
- How to add manual memories and use infer=True to extract memories from conversations
- How to build a scheduled/proactive agent that searches memory, reasons, and acts

**Agent's Goal:** To help a user learn a new skill by remembering their long-term goals, tracking their progress, and proactively suggesting next steps.

**The Flow:**

1. Goal Setting - The user defines their main goal. We manually add this as a core memory.
2. Progress Tracking - The user has a conversation about what they've completed. We use `infer=True` to automatically log this progress.
3. Proactive Agent Logic - We simulate the agent "waking up," reviewing its memory of the user, and generating a personalized, proactive message to help the user take the next step.

Notes:
- Make sure `GRAVIXLAYER_API_KEY` is set in your environment before running the notebook.
- This is intended as a cookbook recipe for Gravix Layer users.

## 1. Setup and Installation

First, we install `gravixlayer` and import our libraries. Remember to set your `GRAVIXLAYER_API_KEY` environment variable and restart your kernel.

In [1]:
!pip install gravixlayer

In [2]:
import os
import time
from gravixlayer import GravixLayer

# Check for API Key
if "GRAVIXLAYER_API_KEY" not in os.environ:
    print("ERROR: GRAVIXLAYER_API_KEY environment variable not set.")
    print("Please set it in your terminal and restart this notebook's kernel.")
else:
    print("✅ GravixLayer API Key found.")

# Initialize the client
client = GravixLayer()
memory = client.memory

# Define a specific user for this agent
LEARNER_USER = "python_learner_88"

✅ GravixLayer API Key found.


## Part 1: Goal Setting (The First Interaction)

The user first tells the agent their main objective. We'll store this as a high-priority, manual memory.

In [3]:
print(f"Setting main goal for user: {LEARNER_USER}...")

main_goal = "The user's primary goal is to learn Python for data science, specifically to get a job in analytics."

memory.add(main_goal, user_id=LEARNER_USER)

print(f"Goal stored: '{main_goal}'")

# Wait for indexing
print("Waiting 2 seconds for goal to index...")
time.sleep(2)

Setting main goal for user: python_learner_88...
Goal stored: 'The user's primary goal is to learn Python for data science, specifically to get a job in analytics.'
Waiting 2 seconds for goal to index...


## Part 2: Progress Tracking (A Later Conversation)

A week later, the user checks in and reports what they've done. We will use `infer=True` to let the agent automatically understand and log this progress.

In [4]:
print("Simulating progress check-in conversation...")

progress_conversation = [
    {"role": "user", "content": "Hey! Just wanted to update you. I finished that 'Intro to Pandas' course you mentioned."}, 
    {"role": "assistant", "content": "That's fantastic progress! Well done."},
    {"role": "user", "content": "It was great, I feel like I understand DataFrames now, but I'm not sure what to learn next."}
]

# Use infer=True to let the AI extract the key facts
result = memory.add(progress_conversation, user_id=LEARNER_USER, infer=True)

print(f"\nAgent extracted {len(result['results'])} new memories from the conversation:")
for mem in result['results']:
    print(f"- {mem['memory']}")

# Wait for indexing
print("\nWaiting 2 seconds for progress to index...")
time.sleep(2)

Simulating progress check-in conversation...

Agent extracted 2 new memories from the conversation:
- Finished 'Intro to Pandas' course.
- Understands DataFrames now.

Waiting 2 seconds for progress to index...


## Part 3: Proactive Agent Logic (The Weekly Check-in)

This is the core of the agent. Imagine this code runs on a schedule (e.g., every Monday).

1.  **Search:** The agent searches its memory for the user's goals and latest progress.
2.  **Think:** It feeds this context into an LLM with a specific system prompt.
3.  **Act:** It generates a proactive, personalized message to coach the user.

In [5]:
def run_proactive_coach_agent(user_id):
    print(f"--- [AGENT WAKES UP] ---")
    print(f"Running proactive check-in for user: {user_id}\n")
    
    # 1. Search: Agent retrieves all relevant memories
    print("[AGENT LOGIC] Searching memory for user's profile...")
    # We use a broad query to get all learning-related info
    memory_results = memory.search("user learning goals and progress", user_id=user_id)
    
    # 2. Think: Agent builds its context and system prompt
    print("[AGENT LOGIC] Building context from memories...")
    context_str = ""
    if memory_results['results']:
        context_str = "Here is what I know about this student:\n"
        for mem in memory_results['results']:
            context_str += f"- {mem['memory']}\n"
    else:
        print("No memories found for user.")
        return
    
    # This is the agent's "personality" and "goal"
    system_prompt = (
        "You are a friendly and proactive AI Learning Coach. "
        "Your goal is to send a single, encouraging check-in message to your student. "
        "Look at their main goal and their most recent progress, then suggest the *next logical step* in their learning journey. "
        "Keep it concise and actionable."
    )
    
    # This is the prompt that *triggers* the agent's action
    user_trigger_prompt = f"Based on this context, write the check-in message for the user.\n\nCONTEXT:\n{context_str}"
    
    print(f"--- DEBUG: Agent is 'thinking' with this prompt --- ")
    print(f"System: {system_prompt}")
    print(f"User/Trigger: {user_trigger_prompt}")
    print("--------------------------------------------------\n")
    
    # 3. Act: Agent generates the proactive message
    print("[AGENT LOGIC] Generating proactive message...")
    response = client.chat.completions.create(
        model="meta-llama/llama-3.1-8b-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_trigger_prompt}
        ]
    )
    
    generated_message = response.choices[0].message.content
    
    print("--- [AGENT ACTION] ---")
    print("Agent sends the following message to the user:\n")
    print(generated_message)
    
# --- Run the Agent ---
run_proactive_coach_agent(LEARNER_USER)

--- [AGENT WAKES UP] ---
Running proactive check-in for user: python_learner_88

[AGENT LOGIC] Searching memory for user's profile...
[AGENT LOGIC] Building context from memories...
--- DEBUG: Agent is 'thinking' with this prompt --- 
System: You are a friendly and proactive AI Learning Coach. Your goal is to send a single, encouraging check-in message to your student. Look at their main goal and their most recent progress, then suggest the *next logical step* in their learning journey. Keep it concise and actionable.
User/Trigger: Based on this context, write the check-in message for the user.

CONTEXT:
Here is what I know about this student:
- The user's primary goal is to learn Python for data science, specifically to get a job in analytics.
- Understands DataFrames now.

--------------------------------------------------

[AGENT LOGIC] Generating proactive message...
--- [AGENT ACTION] ---
Agent sends the following message to the user:

"Hi! Exciting progress with understanding Dat

## 4. Analysis & Next Steps

Notice the output. The agent didn't just say "Hi." It remembered:
1.  **The Goal:** "data science" / "analytics"
2.  **The Progress:** "finished Pandas"
3.  **The Blocker:** "not sure what to learn next"

It correctly suggested that the next logical step after Pandas is often **data visualization** (e.g., Matplotlib or Seaborn), which directly serves the user's main goal.

As a final step, the agent could even store its *own* recommendation in the memory, so it doesn't suggest the same thing next week.

In [8]:
print("Agent is saving its own action to memory...")

agent_action = "Agent proactively suggested the user learn Matplotlib/Seaborn for data visualization next."

memory.add(agent_action, user_id=LEARNER_USER)

print("Action saved. Waiting to index...")
time.sleep(2)

# Now let's see the complete memory log for this user
print("\n--- Complete Memory Log for User ---")

# --- UPDATE ---
# We'll use memory.search() with a broad query, as it proved reliable in the previous step
# (memory.get_all() seems to be returning empty).
print("Using broad search to retrieve all memories...")
all_memories = memory.search("user learning goals and progress", user_id=LEARNER_USER)

if all_memories['results']:
    for i, mem in enumerate(all_memories['results']):
        print(f"{i+1}. {mem['memory']}")
else:
    print("Could not retrieve memories.")

Agent is saving its own action to memory...
Action saved. Waiting to index...

--- Complete Memory Log for User ---
Using broad search to retrieve all memories...
1. Agent proactively suggested the user learn Matplotlib/Seaborn for data visualization next.


## Conclusion

This notebook presented a cookbook example for building a Proactive AI Learning Coach using Gravix Layer's Memory and Chat APIs.

Key takeaways:

- Memory can store explicit user goals and extract facts from conversations (use `infer=True` to automatically extract and store facts).
- The agent pattern (Search → Think → Act) is effective for scheduled, proactive messaging.
- Persisting agent actions back into memory lets the system avoid repeating suggestions and improves personalization over time.

Next steps:

- Add monitoring and logging for production readiness.
- Extend to multi-user scenarios and persistent storage for long-term analysis.
- Add tests for memory indexing and retrieval latency.
